In [10]:
import pandas as pd
import html
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from database import database_manager as dbm

**Extract**

In [11]:
# read in raw data
# file_path = "../../data/raw/covidTwitterData.zip"
# df = pd.read_csv(file_path, compression='zip')
table = "raw_twitter_data"
query = f"""
    SELECT * FROM "{table}"
"""
df = dbm.query_db(query)
display(df)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11.0,110.0,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,Shubham Gupta,None,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17.0,0.0,False,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,Journal of Infectiology,None,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566.0,8.0,False,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,None,Twitter Web App,False
3,Zane,None,Fresher than you.,18-09-2019 11:01,29.0,25.0,620.0,False,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,None,Twitter for Android,False
4,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497.0,10737.0,False,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,None,Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399640,sridhar nandikolla,None,None,2010-05-22 08:23:41,4.0,1,2,false,2022-09-06 04:29:13,#BillGates #BillGatesBioTerrorist \n@BillGates...,"['BillGates', 'BillGatesBioTerrorist', 'COVID1...",Twitter Web App,False
399641,Marc Perreault 🇨🇦,Lacombe,"Husband, dad, follower of Jesus, friend and ho...",2009-04-08 14:13:14,472.0,649,28585,false,2022-09-06 04:25:30,This is incredible. Pfizer boss backtracking o...,"['COVID19', 'CovidVaccine', 'cdnpoli', 'abpoli']",Twitter for Android,False
399642,Christopher Oldcorn,None,Journalist and Opinion Writer @WSOnlineNews co...,2007-09-21 18:02:13,3254.0,4993,8362,false,2022-09-06 04:20:00,#Sask social services admits vaxxing #kids san...,"['Sask', 'kids', 'news', 'canada', 'government...",Buffer,False
399643,Bobby Lee from Tennessee,"Nashville, TN",Nashville Native / Guitar Player / Songwriter ...,2018-03-14 02:56:10,560.0,2353,48738,false,2022-09-06 04:17:25,The pharmaceutical industry would never lie to...,"['COVID', 'Vaccine', 'COVIDvaccine']",Twitter for iPhone,False


**Transform**

In [12]:
# drop all rows containing null
def unescape_html(text): 
    return html.unescape(text)

df_clean = df.dropna().copy()
df_clean.loc[:, 'text'] = (
    df_clean['text']
    .str.replace(r'http\S+|www.\S+', '', regex=True) # remove links
    .str.replace(r'[\n\t]', ' ', regex=True) # remove \t\n
    .apply(unescape_html) # escape html formatting
    .str.replace(r'[^\w\s.,!?;:\-()\'"/]', '', regex=True) # remove non-alphanumeric characters
)

# convert columns to integers
df_clean['user_friends'] = pd.to_numeric(df_clean['user_friends']).replace({False: 0, True: 1}).astype(int)
df_clean['user_followers'] = pd.to_numeric(df_clean['user_followers']).astype(int)
df_clean['user_favourites'] = pd.to_numeric(df_clean['user_favourites']).astype(int)
display(df_clean)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64,11,110,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
7,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,"Iconoclast, cat person, soccer fan, textile & ...",07-02-2015 07:24,2321,3236,264351,False,18-08-2020 12:30,"MSNBC Well, lets qualify that: would anyone of...",['CovidVaccine'],Twitter for iPhone,False
8,Dr. Joseph Santoro,"Washington, DC 20009","Neuro PhD, #Innovator, #Technologist, #Startup...",17-01-2009 21:10,19091,20986,128119,False,18-08-2020 12:15,"Most countries, without the ability to make Va...",['Vaccines'],Hootsuite Inc.,False
9,VUMC OAP,"Nashville, TN","Office of Advanced Practice, Vanderbilt Univer...",16-03-2017 20:22,282,96,788,False,18-08-2020 11:57,DNA zooms up charts in 1st week; hear vaccine...,"['DNA', 'vaccines', 'pandemic', 'COVID19', 'Co...",Twitter Web App,False
11,Mohammadali Naseri,TEHRAN,MSc in Numerical Analysis from the Kharazmi Un...,02-02-2018 17:20,4,88,90,False,18-08-2020 11:04,its over?! Covid19Millionares covid19 corona...,"['Covid19Millionares', 'covid19', 'corona', 'C...",Twitter for iPhone,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399634,MiggyMack 🙂,Johannesburg,Mig-El is my Kryptonian name. \n\nhttps://t.co...,2010-11-27 17:22:10,1083,1481,122176,false,2022-09-06 05:28:32,Whoopsies counts billions he made from goverme...,"['covidvaccine', 'Pfizer']",Twitter for Android,False
399638,CSIR-NEERI: Wastewater Technology Division,Nagpur,To provide solutions and interventions for Ind...,2020-07-07 06:03:00,379,69,387,false,2022-09-06 04:45:37,GIS Representation of Covid_19 scenario for In...,"['GIS', 'Covid_19', 'India', 'Vaccination', 'C...",Twitter Web App,False
399641,Marc Perreault 🇨🇦,Lacombe,"Husband, dad, follower of Jesus, friend and ho...",2009-04-08 14:13:14,472,649,28585,false,2022-09-06 04:25:30,This is incredible. Pfizer boss backtracking o...,"['COVID19', 'CovidVaccine', 'cdnpoli', 'abpoli']",Twitter for Android,False
399643,Bobby Lee from Tennessee,"Nashville, TN",Nashville Native / Guitar Player / Songwriter ...,2018-03-14 02:56:10,560,2353,48738,false,2022-09-06 04:17:25,The pharmaceutical industry would never lie to...,"['COVID', 'Vaccine', 'COVIDvaccine']",Twitter for iPhone,False


In [13]:
print(df_clean['text'])

0         Australia to Manufacture Covid-19 Vaccine and ...
7         MSNBC Well, lets qualify that: would anyone of...
8         Most countries, without the ability to make Va...
9         DNA  zooms up charts in 1st week; hear vaccine...
11        its over?!  Covid19Millionares covid19  corona...
                                ...                        
399634    Whoopsies counts billions he made from goverme...
399638    GIS Representation of Covid_19 scenario for In...
399641    This is incredible. Pfizer boss backtracking o...
399643    The pharmaceutical industry would never lie to...
399644    But their running out of money for the  CovidV...
Name: text, Length: 260213, dtype: object


**Load**

In [ ]:
# file_path = "../../data/processed/cleanCovidTwitterData.zip"
# df_head = df_clean.head()
# df_head.to_csv(file_path, compression='zip', index=False)

"""
Create a new table with the processed data
    dbm.create_table("table_name", dataframe)
"""

"""
Replace a table with the processed data
    dbm.create_table("table_name", dataframe, replace=True)
"""
newTable = "processed_twitter_data"
dbm.create_table(table_name=newTable, dataframe=df_clean, replace=True)

query = f"""
    SELECT * FROM "{newTable}"
"""
df = dbm.query_db(query)
display(df)



,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64,11,110,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,"Iconoclast, cat person, soccer fan, textile & ...",07-02-2015 07:24,2321,3236,264351,False,18-08-2020 12:30,"MSNBC Well, lets qualify that: would anyone of...",['CovidVaccine'],Twitter for iPhone,False
2,Dr. Joseph Santoro,"Washington, DC 20009","Neuro PhD, #Innovator, #Technologist, #Startup...",17-01-2009 21:10,19091,20986,128119,False,18-08-2020 12:15,"Most countries, without the ability to make Va...",['Vaccines'],Hootsuite Inc.,False
3,VUMC OAP,"Nashville, TN","Office of Advanced Practice, Vanderbilt Univer...",16-03-2017 20:22,282,96,788,False,18-08-2020 11:57,DNA zooms up charts in 1st week; hear vaccine...,"['DNA', 'vaccines', 'pandemic', 'COVID19', 'Co...",Twitter Web App,False
4,Mohammadali Naseri,TEHRAN,MSc in Numerical Analysis from the Kharazmi Un...,02-02-2018 17:20,4,88,90,False,18-08-2020 11:04,its over?! Covid19Millionares covid19 corona...,"['Covid19Millionares', 'covid19', 'corona', 'C...",Twitter for iPhone,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260208,MiggyMack 🙂,Johannesburg,Mig-El is my Kryptonian name. \n\nhttps://t.co...,2010-11-27 17:22:10,1083,1481,122176,false,2022-09-06 05:28:32,Whoopsies counts billions he made from goverme...,"['covidvaccine', 'Pfizer']",Twitter for Android,False
260209,CSIR-NEERI: Wastewater Technology Division,Nagpur,To provide solutions and interventions for Ind...,2020-07-07 06:03:00,379,69,387,false,2022-09-06 04:45:37,GIS Representation of Covid_19 scenario for In...,"['GIS', 'Covid_19', 'India', 'Vaccination', 'C...",Twitter Web App,False
260210,Marc Perreault 🇨🇦,Lacombe,"Husband, dad, follower of Jesus, friend and ho...",2009-04-08 14:13:14,472,649,28585,false,2022-09-06 04:25:30,This is incredible. Pfizer boss backtracking o...,"['COVID19', 'CovidVaccine', 'cdnpoli', 'abpoli']",Twitter for Android,False
260211,Bobby Lee from Tennessee,"Nashville, TN",Nashville Native / Guitar Player / Songwriter ...,2018-03-14 02:56:10,560,2353,48738,false,2022-09-06 04:17:25,The pharmaceutical industry would never lie to...,"['COVID', 'Vaccine', 'COVIDvaccine']",Twitter for iPhone,False
